In [24]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [25]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

jobs_to_nodes_whole_data contains 469095 records
There are 35284 long jobs in the Andrea's interval


#### solve the problem of jobs runned on node 129

In [26]:
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

after the clean, jobs_to_nodes_whole_data contains 444610 records


In [27]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(long_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 41216 records


In [28]:
merged_jobs_to_nodes.head()

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,node_id,ncpus,ngpus,nmics,mem_requested
0,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,True,0.000000,0.0,1.0,0.0,17,16,0,0,14680064
1,498459.node129,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo@node129.eurora.cineca.it,1,16,14,...,True,0.974359,1.0,0.0,0.0,9,16,0,0,14680064
2,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,True,0.880787,5.0,1.0,0.0,9,16,0,0,14680064
3,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,True,0.880787,5.0,1.0,0.0,28,16,0,0,14680064
4,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0@node129.eurora.cineca.it,6,96,84,...,True,0.880787,5.0,1.0,0.0,25,16,0,0,14680064


In [29]:
#short_jobs = pd.read_csv(infile_short_jobs + ".csv", index_col=0)
#print("There are {} long jobs in the {}'s interval".format(short_jobs.shape[0], interval_comment))

In [30]:
merged_jobs_to_nodes.shape[0]

41216

In [32]:
for node in nodes:
    #print(node)
    jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    #print(jobs_to_node.shape[0])
    mean_core_consumption = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    jobs_to_node = jobs_to_node[jobs_to_node['real_pow_quality'] != 0]
    mean_core_consumption_medium = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    jobs_to_node = jobs_to_node[jobs_to_node['real_pow_quality'] >= 0.5]
    mean_core_consumption_high = jobs_to_node['real_pow'].sum() / jobs_to_node['cpu_req'].sum()
    print("{}, {}, {}, {}".format(node, mean_core_consumption, mean_core_consumption_medium, mean_core_consumption_high))

01, 9.303076930042387, 10.074514527415673, 11.298611600237855
02, 7.697023455152835, 7.697023455152835, 8.333598124765686
03, 13.797005282298862, 13.797005282298862, 13.835960591081806
04, 13.93442842693612, 13.93442842693612, 13.93442842693612
05, 13.255770378097182, 13.255770378097182, 13.321348299480992
06, 12.086335971411629, 12.090269170316917, 12.642440504873887
07, 12.968377460177507, 12.9932051369709, 13.075835070257462
08, 13.021782963872438, 13.034363991131361, 13.563582129415245
09, 11.193661332340675, 11.219268561871193, 11.696159419999914
10, 11.668800701842088, 11.699855851828849, 12.067063157530551
11, 12.710658027501301, 12.743872513783613, 12.73582842592844
12, 12.046880288668913, 12.065688896672729, 12.447110169968905
13, 11.88099906422749, 11.907813454714653, 12.352682667831084
14, 12.37029828790492, 12.393079155518553, 12.823573852862383
15, 11.422261817159924, 11.446567213702744, 11.744760800577009
16, 11.23157027230262, 11.23157027230262, 11.567829243659743
17, 17

## Prediction

In [46]:
from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error

infile_mean_node_consumptions = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_perNode"
mean_node_consumptions = pd.read_csv(infile_mean_node_consumptions + ".csv")

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))
long_jobs.head(1)

There are 35284 long jobs in the Andrea's interval


,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,exit_status,pbs_exit_code,gpu_req,mic_req,real_pow,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1
job_id,,,,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,COMPLETED,NaN,NaN,NaN,140.0,True,0.0,0.0,1.0,0.0


In [50]:
mean_node_consumptions.head(1)

,node,all,medium,high
0,1,9.303077,10.074515,11.298612


In [47]:
merged_jobs_to_nodes.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,node_id,ncpus,ngpus,nmics,mem_requested
0,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,...,True,0.0,0.0,1.0,0.0,17,16,0,0,14680064


In [83]:
long_jobs['pred_perNode'] = 0.0
i = 0

for index_job, job in long_jobs.iterrows():

    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job['job_id_string']] 
    job_predicted_consumption = 0.0
    
    for index_node, job_to_node in job_to_nodes.iterrows():
        
        mean_core_consumptions = mean_node_consumptions[mean_node_consumptions['node'] == job_to_node['node_id']].iloc[0][1]
        job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumptions
        
    long_jobs.at[index_job, 'pred_perNode'] = job_predicted_consumption
    if (i % 10000 == 0):
        print(i)
    i += 1

slope, intercept, r_value, p_value, std_err = stats.linregress(long_jobs['real_pow'], long_jobs['pred_perNode'])
print("R-squared: {}".format(r_value**2))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


In [86]:
long_jobs = long_jobs[long_jobs['real_pow_quality'] > 0]

In [88]:
long_jobs['pred_perNode'] = 0.0
i = 0

for index_job, job in long_jobs.iterrows():

    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job['job_id_string']] 
    job_predicted_consumption = 0.0
    
    for index_node, job_to_node in job_to_nodes.iterrows():
        
        mean_core_consumptions = mean_node_consumptions[mean_node_consumptions['node'] == job_to_node['node_id']].iloc[0][2]
        job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumptions
        
    long_jobs.at[index_job, 'pred_perNode'] = job_predicted_consumption
    if (i % 10000 == 0):
        print(i)
    i += 1

slope, intercept, r_value, p_value, std_err = stats.linregress(long_jobs['real_pow'], long_jobs['pred_perNode'])
print("R-squared: {}".format(r_value**2))

0
10000
20000
30000
R-squared: 0.7290160023770573


In [89]:
from sklearn.metrics import mean_squared_error
print("mean_quared_error: {}".format(mean_squared_error(long_jobs['real_pow'], long_jobs['pred_perNode'])))

mean_quared_error: 20227.450851249076
